In [1]:
import numpy as np
import pandas as pd
pd.set_option('max_columns', None)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [56]:
#url = 'https://www.kaggle.com/pavansubhasht/ibm-hr-analytics-attrition-dataset'
#data = pd.read_csv(url)
#from google.colab import files
#uploaded = files.upload()

Saving emp_test.csv to emp_test.csv


In [4]:
data = pd.read_csv('emp_train.csv')

In [5]:
data

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,1,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,49,0,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
2,37,1,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0
3,33,0,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
4,27,0,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053,57,0,Travel_Rarely,405,Research & Development,1,2,Life Sciences,1,1483,2,Male,93,4,2,Research Scientist,3,Married,4900,2721,0,Y,No,24,4,1,80,1,13,2,2,12,9,2,8
1054,49,0,Travel_Rarely,1490,Research & Development,7,4,Life Sciences,1,1484,3,Male,35,3,3,Healthcare Representative,2,Divorced,10466,20948,3,Y,No,14,3,2,80,2,29,3,3,8,7,0,7
1055,34,0,Travel_Frequently,829,Research & Development,15,3,Medical,1,1485,2,Male,71,3,4,Research Director,1,Divorced,17007,11929,7,Y,No,14,3,4,80,2,16,3,2,14,8,6,9
1056,28,1,Travel_Frequently,1496,Sales,1,3,Technical Degree,1,1486,1,Male,92,3,1,Sales Representative,3,Married,2909,15747,3,Y,No,15,3,4,80,1,5,3,4,3,2,1,2


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1058 entries, 0 to 1057
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1058 non-null   int64 
 1   Attrition                 1058 non-null   int64 
 2   BusinessTravel            1058 non-null   object
 3   DailyRate                 1058 non-null   int64 
 4   Department                1058 non-null   object
 5   DistanceFromHome          1058 non-null   int64 
 6   Education                 1058 non-null   int64 
 7   EducationField            1058 non-null   object
 8   EmployeeCount             1058 non-null   int64 
 9   EmployeeNumber            1058 non-null   int64 
 10  EnvironmentSatisfaction   1058 non-null   int64 
 11  Gender                    1058 non-null   object
 12  HourlyRate                1058 non-null   int64 
 13  JobInvolvement            1058 non-null   int64 
 14  JobLevel                

In [7]:
print(data.shape)
{col: len(data[col].unique()) for col in data.columns}

(1058, 35)


{'Age': 43,
 'Attrition': 2,
 'BusinessTravel': 3,
 'DailyRate': 731,
 'Department': 3,
 'DistanceFromHome': 29,
 'Education': 5,
 'EducationField': 6,
 'EmployeeCount': 1,
 'EmployeeNumber': 1058,
 'EnvironmentSatisfaction': 4,
 'Gender': 2,
 'HourlyRate': 71,
 'JobInvolvement': 4,
 'JobLevel': 5,
 'JobRole': 9,
 'JobSatisfaction': 4,
 'MaritalStatus': 3,
 'MonthlyIncome': 991,
 'MonthlyRate': 1038,
 'NumCompaniesWorked': 10,
 'Over18': 1,
 'OverTime': 2,
 'PercentSalaryHike': 15,
 'PerformanceRating': 2,
 'RelationshipSatisfaction': 4,
 'StandardHours': 1,
 'StockOptionLevel': 4,
 'TotalWorkingYears': 40,
 'TrainingTimesLastYear': 7,
 'WorkLifeBalance': 4,
 'YearsAtCompany': 35,
 'YearsInCurrentRole': 19,
 'YearsSinceLastPromotion': 16,
 'YearsWithCurrManager': 18}

In [9]:
def tr2(df):
  df = df.copy()
  for col in df.columns:
    if(df[col].dtype == np.number):
      continue
    df[col] = LabelEncoder().fit_transform(df[col])
  return df

In [10]:
def preprocess_inputs(df):
    df = df.copy()
    
    # Drop single-value columns and id columns
    df = df.drop(['EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours'], axis=1)

    # Binary-encode binary columns
    df['Gender'] = df['Gender'].replace({'Female': 0, 'Male': 1})
    df['OverTime'] = df['OverTime'].replace({'No': 0, 'Yes': 1})
      
    # Ordinal-encode the BusinessTravel column
    df['BusinessTravel'] = df['BusinessTravel'].replace({'Non-Travel': 0, 'Travel_Rarely': 1, 'Travel_Frequently': 2})
    
    # One-hot encoding
    #for column in ['Department', 'EducationField', 'JobRole', 'MaritalStatus']:
        #df = onehot_encode(df, column=column)
    
    df = tr2(df)
    
    # Split df into X and y
    y = df['Attrition']
    X = df.drop('Attrition', axis=1)
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)
    
    # Scale X
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)
    
    return X_train, X_test, y_train, y_test,scaler

In [26]:
def op_preprocess(sample,scaler):
    df = sample.copy()
    
    # Drop single-value columns and id columns
    df = df.drop(['EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours'], axis=1)

    # Binary-encode binary columns
    df['Gender'] = df['Gender'].replace({'Female': 0, 'Male': 1})
    df['OverTime'] = df['OverTime'].replace({'No': 0, 'Yes': 1})
      
    # Ordinal-encode the BusinessTravel column
    df['BusinessTravel'] = df['BusinessTravel'].replace({'Non-Travel': 0, 'Travel_Rarely': 1, 'Travel_Frequently': 2})
    
    # One-hot encoding
    #for column in ['Department', 'EducationField', 'JobRole', 'MaritalStatus']:
        #df = onehot_encode(df, column=column)

    df = tr2(df)
    
    #print("Inside==============================================")
    #print(df.shape)
    
    # Split df into X and y
    #y = df['Attrition']
    #X = df.drop('Attrition', axis=1)
    
    # Train-test split
    #X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)
    
    # Scale X
    #scaler = StandardScaler()
    #scaler.fit(X_train)
    #X_test = pd.DataFrame(scaler.transform(df), index=df.index, columns=df.columns)
    #X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)

    df = pd.DataFrame(scaler.transform(df), index=df.index, columns=df.columns)
    
    return df

In [11]:
X_train, X_test, y_train, y_test,scaler = preprocess_inputs(data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  after removing the cwd from sys.path.


In [12]:
X_train

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
69,-0.151184,-0.178690,-1.210770,-0.516815,-0.001490,0.049628,0.543481,1.171467,0.877687,0.668363,-1.036543,-0.989936,0.619117,0.239205,-0.133876,-0.721186,1.084744,-1.097594,1.583385,0.544472,-0.413384,-1.606612,0.297044,-1.207653,-2.230260,-1.080579,-1.003995,-1.195867,-0.693579,-1.169939
491,0.589717,1.710319,0.451045,-0.516815,-0.001490,2.034728,0.543481,1.171467,0.877687,0.320760,0.399865,-0.092200,-1.040835,0.239205,-1.490975,0.259063,0.775315,0.071067,1.583385,-0.563944,-0.413384,-0.684989,0.297044,-0.219361,0.140993,0.337202,0.146601,0.703732,0.493122,0.770594
1024,1.013090,-0.178690,-1.130054,-0.516815,-0.858797,1.042178,0.543481,-1.574739,-1.139358,0.817335,0.399865,1.703272,0.204129,0.239205,-0.133876,1.504433,1.726894,0.071067,-0.631558,1.098680,-0.413384,-0.684989,1.518226,1.757223,-0.649424,1.754983,2.119052,3.417444,0.789798,0.493375
781,-1.209614,-0.178690,-1.514645,-0.516815,-0.981270,-0.942923,0.543481,-1.574739,0.877687,-0.324787,-1.036543,-0.989936,-1.040835,-1.557871,-0.133876,-0.504903,-0.389206,-0.708040,-0.631558,0.267368,-0.413384,-1.606612,1.518226,-0.713507,-0.649424,0.337202,-0.346512,-0.381753,-0.396903,-0.338282
757,-0.362870,-0.178690,-1.448173,1.385888,-0.981270,1.042178,-0.205450,-0.659337,0.877687,0.469733,1.836274,-0.092200,1.034105,1.137744,-1.490975,0.956749,-0.222846,-1.097594,-0.631558,-1.118153,-0.413384,1.158255,0.297044,0.521858,-0.649424,-1.080579,1.297198,-0.924496,-0.693579,1.325032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960,0.378031,1.710319,0.489029,1.385888,-0.981270,0.049628,-0.205450,0.256065,-1.139358,0.022816,0.399865,-0.092200,1.034105,-1.557871,-1.490975,-0.452577,-1.384040,-1.097594,-0.631558,0.544472,-0.413384,1.158255,0.297044,-0.219361,-0.649424,0.337202,0.310972,-0.381753,-0.396903,0.770594
847,-0.362870,1.710319,-0.341878,-0.516815,-0.981270,0.049628,0.543481,1.171467,0.877687,1.562197,-1.036543,-0.092200,-1.870811,-1.557871,1.223223,0.147433,1.643714,-1.097594,-0.631558,1.375785,2.419060,0.236633,-0.924138,0.274785,0.140993,0.337202,0.968456,1.246474,0.493122,1.325032
715,-0.468713,1.710319,-0.000019,-0.516815,-0.981270,1.042178,1.292413,0.256065,-1.139358,0.916650,1.836274,-0.092200,-1.870811,-0.659333,-0.133876,0.217202,0.855167,-0.708040,1.583385,-0.563944,-0.413384,-1.606612,0.297044,-0.713507,-0.649424,0.337202,-0.182141,0.160989,-0.396903,-0.615501
905,-0.892085,-0.178690,-0.270658,-0.516815,-0.981270,0.049628,-0.954382,1.171467,-1.139358,1.065622,-1.036543,1.703272,0.204129,1.137744,-1.490975,1.413734,-1.497164,0.071067,-0.631558,-0.286840,-0.413384,-0.684989,1.518226,-0.342897,-0.649424,-1.080579,-0.017770,0.703732,-0.396903,0.770594


In [13]:
y_train

69      1
491     0
1024    0
781     0
757     0
       ..
960     0
847     0
715     0
905     0
235     0
Name: Attrition, Length: 740, dtype: int64

**Training**

In [71]:
models = {
    "                   Logistic Regression": LogisticRegression(),
    "                   K-Nearest Neighbors": KNeighborsClassifier(),
    "                         Decision Tree": DecisionTreeClassifier(),
    "Support Vector Machine (Linear Kernel)": LinearSVC(max_iter=1500),
    "   Support Vector Machine (RBF Kernel)": SVC(),
    "                        Neural Network": MLPClassifier(max_iter=1500),
    "                         Random Forest": RandomForestClassifier(),
    "                     Gradient Boosting": GradientBoostingClassifier()
}

#pre_models = models.copy()

print("Normalized Dataset")
for name, model in models.items():
    model.fit(X_train, y_train)
    print(name + " trained.")
print('--------------------------------')
'''
print("Handpicked Dataset")
for name, pre_model in pre_models.items():
    pre_model.fit(pre_X_train, pre_y_train)
    print(name + " trained.")'''

Normalized Dataset
                   Logistic Regression trained.
                   K-Nearest Neighbors trained.
                         Decision Tree trained.
Support Vector Machine (Linear Kernel) trained.
   Support Vector Machine (RBF Kernel) trained.
                        Neural Network trained.
                         Random Forest trained.
                     Gradient Boosting trained.
--------------------------------


'\nprint("Handpicked Dataset")\nfor name, pre_model in pre_models.items():\n    pre_model.fit(pre_X_train, pre_y_train)\n    print(name + " trained.")'

Output

In [72]:
print("Normalized dataset")
for name, model in models.items():
  print(name + ": {:.2f}%".format(model.score(X_test, y_test) * 100))
print("------------------------------")
'''
print("Handpicked Dataset")
for name, pre_model in pre_models.items():
    print(name + ": {:.2f}%".format(pre_model.score(pre_X_test, pre_y_test) * 100))'''

Normalized dataset
                   Logistic Regression: 85.85%
                   K-Nearest Neighbors: 82.39%
                         Decision Tree: 77.04%
Support Vector Machine (Linear Kernel): 86.79%
   Support Vector Machine (RBF Kernel): 82.70%
                        Neural Network: 84.59%
                         Random Forest: 82.08%
                     Gradient Boosting: 81.45%
------------------------------


'\nprint("Handpicked Dataset")\nfor name, pre_model in pre_models.items():\n    print(name + ": {:.2f}%".format(pre_model.score(pre_X_test, pre_y_test) * 100))'

In [73]:
svm = LinearSVC(max_iter=1500)

In [74]:
svm.fit(X_train,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1500,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [75]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
pred = svm.predict(X_test)

cm = confusion_matrix(y_test,pred)
print(cm)

print("Accuracy:", accuracy_score(y_test,pred))
print(classification_report(y_test,pred))

[[248   9]
 [ 33  28]]
Accuracy: 0.8679245283018868
              precision    recall  f1-score   support

           0       0.88      0.96      0.92       257
           1       0.76      0.46      0.57        61

    accuracy                           0.87       318
   macro avg       0.82      0.71      0.75       318
weighted avg       0.86      0.87      0.85       318



In [38]:
df = data.iloc[0:2,:]
print(df['Attrition'])
df = df.drop('Attrition', axis = 1)

#print("Dataframe\n",df)
#df = np.array(df)

#print(df.shape)

#smple = pd.DataFrame(df,columns=data.columns)
#smple

#smple.drop('Attrition',axis = 1)

#print(smple.shape)

df = op_preprocess(df,scaler)

#print("Dataframe\n",df)

#print(df.shape)

#model["Support Vector Machine (Linear Kernel)"].predict(smple)

0    1
1    0
Name: Attrition, dtype: int64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  after removing the cwd from sys.path.


In [76]:
df = pd.read_csv('emp_test.csv')
df = op_preprocess(df,scaler)
pred = svm.predict(df)
pred

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  after removing the cwd from sys.path.


array([1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,

In [77]:
#df = data.iloc[0:2,:]
df = pd.read_csv('emp_test.csv')
#print(df['Attrition'])
#df = df.drop('Attrition', axis = 1)
df = op_preprocess(df,scaler)
for row in range(df.shape[0]):
  for (intercept, coef) in zip(svm.intercept_, svm.coef_):
      s = "y = {0:.3f}".format(intercept)
      h = intercept
      mx = -1
      for (i, c) in enumerate(coef):
          s += " + {0:.3f} * x{1}".format(c, i)
          k = max(h,c*df.iloc[row][i])
          if(k>h):
            h = k
            mx = i
      #print(s)
      print(h,mx)
      print(pred[row],df.columns[mx])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  after removing the cwd from sys.path.


0.463915114434167 18
1 OverTime
0.450364313611681 15
0 MonthlyIncome
0.463915114434167 18
1 OverTime
0.48604792014774884 15
0 MonthlyIncome
0.463915114434167 18
1 OverTime
0.18174199372570868 3
0 Department
0.481969793686484 15
0 MonthlyIncome
0.463915114434167 18
0 OverTime
0.3616650630791695 15
0 MonthlyIncome
0.463915114434167 18
0 OverTime
0.4350713393819377 15
1 MonthlyIncome
0.4942041730702787 15
0 MonthlyIncome
0.29845410292956365 15
0 MonthlyIncome
0.30559082423677725 15
0 MonthlyIncome
0.38511429023144267 15
0 MonthlyIncome
0.33469657567263844 10
0 JobInvolvement
0.463915114434167 18
0 OverTime
0.1580058970151606 14
0 MaritalStatus
0.29842033216637254 11
0 JobLevel
0.45546197168826213 15
1 MonthlyIncome
0.5640103947859156 28
0 YearsSinceLastPromotion
0.17814937232224917 15
0 MonthlyIncome
0.3430427440649067 17
0 NumCompaniesWorked
0.18525430014257946 25
0 WorkLifeBalance
0.4656572878414244 15
1 MonthlyIncome
0.4880869833783813 15
0 MonthlyIncome
0.463915114434167 18
1 OverTime

In [78]:
import pickle
# open a file, where you ant to store the data
file = open('flight_rf.pkl', 'wb')

# dump information to that file
pickle.dump(svm, file)